<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing required libraries
!pip install torchinfo
import torchinfo
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [76]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [93]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.conv1(x)

    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)



    return x



model = LeNet5().to(device)
torch.nn.init.xavier_uniform_(model.conv1.weight)
torch.nn.init.xavier_uniform_(model.conv2.weight)



Parameter containing:
tensor([[[[ 8.3098e-02, -3.4200e-02, -5.4602e-04,  9.3898e-03,  6.4298e-02],
          [ 5.1020e-03, -7.6593e-02, -2.6232e-02,  3.0386e-02, -4.5831e-02],
          [-3.4852e-02, -9.0945e-02,  1.0091e-01, -1.0080e-01, -5.6156e-02],
          [ 8.2539e-02, -7.4285e-02,  5.4630e-02, -9.4860e-02, -1.0237e-01],
          [-9.4343e-02,  9.2724e-02, -4.5739e-02, -6.7224e-02,  8.2351e-02]],

         [[ 3.1137e-02, -5.7217e-02,  3.6438e-02,  4.8372e-02, -8.6419e-02],
          [ 1.4650e-02,  3.4551e-02, -6.9161e-02, -6.9668e-02, -8.7025e-02],
          [ 4.1072e-02, -9.5803e-02,  9.2088e-02, -5.8041e-02,  6.7873e-02],
          [-4.3564e-02, -9.7144e-03, -8.7736e-02, -9.5103e-02,  5.8194e-02],
          [ 1.3696e-02,  1.0417e-01,  4.1869e-02,  1.0019e-01,  6.9932e-02]],

         [[ 1.8437e-02, -6.1176e-02,  8.0624e-02,  7.1979e-02,  9.2771e-02],
          [-2.9149e-02,  3.9104e-02, -6.8021e-02, -6.3580e-02, -7.3332e-02],
          [ 7.3564e-02, -3.7582e-02,  3.0900e-03, 

In [94]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.RandomCrop(32, padding=4), transforms.ToTensor(), transforms.Normalize(mean=0,std=1)])
train_data = CIFAR10(root="./train/", train=False, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data,batch_size=16, shuffle=True)

Files already downloaded and verified


In [95]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(), lr=1)


In [ ]:
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)


    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      #global epoch_loss

      return loss
    optimizer.step(closure)
    loss = closure()
    epoch_loss += loss.item()

  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))

